### Imports

In [1]:
import numpy as np
import pandas as pd
import arviz as az
from cmdstanpy import CmdStanModel

## Regression Models

Stan supports regression models from simple linear regressions to multilevel generalized linear models.

### Linear regression

The simplest linear regression model is the following, with a single predictor and a slope and intercept coefficient, and normally distributed noise. This model can be written using standard regression notation as:
$$
y_n = \alpha + \beta x_n + \epsilon_n
\quad\text{where}\quad
\epsilon_n \sim \operatorname{normal}(0,\sigma).
$$

This is equivalent to the following sampling involving the residual,
$$
y_n - (\alpha + \beta X_n) \sim \operatorname{normal}(0,\sigma),
$$
and reducing still further, to
$$
y_n \sim \operatorname{normal}(\alpha + \beta X_n, \, \sigma).
$$

In [6]:
linear_code_1 = '''
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] y;
}

parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}

model {
    y ~ normal(alpha + beta * x, sigma);
}
'''

stan_file = './stan_models/linear_code_1.stan'

with open(stan_file, 'w') as f:
    print(linear_code_1, file=f)
    
linear_1_model = CmdStanModel(stan_file=stan_file, force_compile=True, cpp_options={'STAN_THREADS':'true'})

16:32:25 - cmdstanpy - INFO - compiling stan file /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/linear_code_1.stan to exe file /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/linear_code_1
16:32:35 - cmdstanpy - INFO - compiled model executable: /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/linear_code_1


There are `N` observations and for each observation, $n \in N$,  we have predictor `x[n]` and outcome `y[n]`.  The intercept and slope parameters are `alpha` and `beta`. The model assumes a normally
distributed noise term with scale `sigma`. This model has improper priors for the two regression coefficients.

#### Matrix notation and vectorization

The distribution statement in the previous model is vectorized, with

```stan
y ~ normal(alpha + beta * x, sigma);
```

providing the same model as the unvectorized version,

```stan
for (n in 1:N) {
  y[n] ~ normal(alpha + beta * x[n], sigma);
}
```

In addition to being more concise, the vectorized form is much faster.

In general, Stan allows the arguments to distributions such as `normal` to be vectors. If any of the other arguments are vectors or arrays, they have to be the same size. If any of the other arguments is a scalar, it is reused (or broadcasted) for each vector entry.

The other reason this works is that Stan's arithmetic operators are overloaded to perform matrix arithmetic on matrices.  In this case, because `x` is of type `vector` and `beta` of type `real`, the expression `beta * x` is of type `vector`. Because Stan supports vectorization, a regression model with more than one predictor can be written directly using matrix notation.

In [3]:
linear_code_2 = '''
data {
    int<lower=0> N;         // number of data items
    int<lower=0> K;         // number of predictors
    matrix[N, K] x;         // predictor matrix
    vector[N] y;            // outcome vector
}

parameters {
    real alpha;             // intercept
    vector[K] beta;         // coefficients for predictors
    real<lower=0> sigma;    // error scale
}

model {
    y ~ normal(x * beta + alpha, sigma); // data model
}
'''

stan_file = './stan_models/linear_code_2.stan'

with open(stan_file, 'w') as f:
    print(linear_code_2, file=f)
    
linear_2_model = CmdStanModel(stan_file=stan_file, force_compile=True)

16:28:12 - cmdstanpy - INFO - compiling stan file /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/linear_code_2.stan to exe file /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/linear_code_2
16:28:21 - cmdstanpy - INFO - compiled model executable: /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/linear_code_2


The constraint `lower=0` in the declaration of `sigma` constrains the value to be greater than or equal to 0.  With no prior in the model block, the effect is an improper prior on non-negative real numbers.  Although a more informative prior may be added, improper priors are acceptable as long as they lead to proper posteriors.

In the model above, `x` is an $N \times K$ matrix of predictors and `beta` a $K$-vector of coefficients, so `x * beta` is an $N$-vector of predictions, one for each of the $N$ data items. These
predictions line up with the outcomes in the $N$-vector `y`, so the entire model may be written using matrix arithmetic as shown.  It would be possible to include a column of ones in the data matrix `x` to
remove the `alpha` parameter.

The distribution statement in the model above is just a more efficient, vector-based approach to coding the model with a loop, as in the following statistically equivalent model.

```stan
model {
  for (n in 1:N) {
    y[n] ~ normal(x[n] * beta, sigma);
  }
}
```

With Stan's matrix indexing scheme, `x[n]` picks out row `n` of the matrix `x`;  because `beta` is a column vector, the product `x[n] * beta` is a scalar of type `real`.

##### Intercepts as inputs

In the model formulation

```stan
y ~ normal(x * beta, sigma);
```

there is no longer an intercept coefficient `alpha`.  Instead, we have assumed that the first column of the input matrix `x` is a column of 1 values.  This way, `beta[1]` plays the role of the intercept.  If the intercept gets a different prior than the slope terms, then it would be clearer to break it out.  It is also slightly more efficient in its explicit form with the intercept variable
singled out because there's one fewer multiplications; it should not make that much of a difference to speed, though, so the choice should be based on clarity.

### The QR reparameterization

In the previous example, the linear predictor can be written as $\eta = x \beta$, where $\eta$ is a $N$-vector of predictions, $x$ is a $N \times K$ matrix, and $\beta$ is a $K$-vector of coefficients.
Presuming $N \geq K$, we can exploit the fact that any design matrix $x$ can be decomposed using the thin QR decomposition into an orthogonal matrix $Q$ and an upper-triangular matrix $R$, i.e. $x = Q
R$.

The functions `qr_thin_Q` and `qr_thin_R` implement the thin QR decomposition, which is to be preferred to the fat QR decomposition that would be obtained by using `qr_Q` and `qr_R`, as the latter would more easily run out of memory (see the Stan Functions Reference for more information on the `qr_thin_Q` and `qr_thin_R` functions). In practice, it is best to write $x = Q^\ast R^\ast$ where $Q^\ast = Q * \sqrt{n - 1}$ and $R^\ast = \frac{1}{\sqrt{n - 1}} R$. Thus, we can equivalently write $\eta = x \beta = Q R \beta = Q^\ast R^\ast \beta$. If we let $\theta = R^\ast \beta$, then we have $\eta = Q^\ast \theta$ and $\beta = R^{\ast^{-1}} \theta$. In that case, the previous Stan program becomes

In [13]:
qr_reparam = '''
data {
    int<lower=0> N;         // number of data items
    int<lower=0> K;         // number of predictors
    matrix[N, K] x;         // predictor matrix
    vector[N] y;            // outcome vector
}

transformed data {
    matrix[N, K] Q_ast;
    matrix[K, K] R_ast;
    matrix[K, K] R_ast_inverse;
    // thin and scale the QR decomposition
    Q_ast = qr_thin_Q(x) * sqrt(N - 1);
    R_ast = qr_thin_R(x) / sqrt(N - 1);
    R_ast_inverse = inverse(R_ast);
}

parameters {
    real alpha;             // intercept
    vector[K] theta;        // coefficients for Q_ast
    real<lower=0> sigma;    // error scale    
}

model {
    y ~ normal(Q_ast * theta + alpha, sigma); // data model
}

generated quantities {
    vector[K] beta;
    beta = R_ast_inverse * theta; // coefficients on x
}
'''

stan_file = './stan_models/qr_reparam.stan'

with open(stan_file, 'w') as f:
    print(qr_reparam, file=f)
    
qr_reparam_model = CmdStanModel(stan_file=stan_file, force_compile=True)

17:20:59 - cmdstanpy - INFO - compiling stan file /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/qr_reparam.stan to exe file /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/qr_reparam
17:21:10 - cmdstanpy - INFO - compiled model executable: /Users/rehabnaeem/Documents/Coding-Projects/bayesian-analysis/references/Stan-Modelling/stan_models/qr_reparam


Since this Stan program generates equivalent predictions for $y$ and the same posterior distribution for $\alpha$, $\beta$, and $\sigma$ as the previous Stan program, many wonder why the version with this QR reparameterization performs so much better in practice, often both in terms of wall time and in terms of effective sample size. The reasoning is threefold:

1. The columns of $Q^\ast$ are orthogonal whereas the columns of $x$ generally are not. Thus, it is easier for a Markov Chain to move around in $\theta$-space than in $\beta$-space.
2. The columns of $Q^\ast$ have the same scale whereas the columns of $x$ generally do not. Thus, a Hamiltonian Monte Carlo algorithm can move around the parameter space with a smaller number of larger steps
3. Since the covariance matrix for the columns of $Q^\ast$ is an identity matrix, $\theta$ typically has a reasonable scale if the units of $y$ are also reasonable. This also helps HMC move efficiently without compromising numerical accuracy.

Consequently, this QR reparameterization is recommended for linear and generalized linear models in Stan whenever $K > 1$ and you do not have an informative prior on the location of $\beta$. It can also be worthwhile to subtract the mean from each column of $x$ before obtaining the QR decomposition, which does not affect the posterior distribution of $\theta$ or $\beta$ but does affect $\alpha$ and
allows you to interpret $\alpha$ as the expectation of $y$ in a linear model.